In [ ]:
# Challenge 1: Fork Languages

In [1]:
import requests 

In [20]:

from urllib.parse import urlparse

def get_fork_languages(repo_url):
 
    parsed_url = urlparse(repo_url)
    username = parsed_url.path.split('/')[1]

    
    forks_url = f"https://api.github.com/users/{username}/repos"
    
    response = requests.get(forks_url)
    repositories = response.json()

    if not repositories:
        print(f"No repositories found for {username}")
        return

    languages = set()
    
    for repo in repositories:
        repo_name = repo.get('name')
        fork_languages = get_repository_languages(username, repo_name)
        
        print(f"Languages for fork {username}/{repo_name}: {fork_languages}")

def get_repository_languages(username, repo):
    api_url = f"https://api.github.com/repos/{username}/{repo}/languages"
    
    response = requests.get(api_url)
    languages = response.json()

    return list(languages.keys())

github_profile_url = "https://github.com/TMeesters?tab=repositories"


get_fork_languages(github_profile_url)


Languages for fork TMeesters/-lab-api-scavenger: ['Jupyter Notebook']
Languages for fork TMeesters/Descriptive-Stats: ['Jupyter Notebook']
Languages for fork TMeesters/lab-advanced-pandas: ['Jupyter Notebook']
Languages for fork TMeesters/lab-bash: []
Languages for fork TMeesters/lab-data-cleaning: ['Jupyter Notebook']
Languages for fork TMeesters/lab-dataframe-calculations: ['Jupyter Notebook']
Languages for fork TMeesters/lab-dw-data-cleaning-and-formatting: ['Jupyter Notebook']
Languages for fork TMeesters/lab-dw-pandas: ['Jupyter Notebook']
Languages for fork TMeesters/lab-git: ['Jupyter Notebook']
Languages for fork TMeesters/lab-import-export: ['Jupyter Notebook']
Languages for fork TMeesters/lab-intro-pandas: ['Jupyter Notebook']
Languages for fork TMeesters/lab-lambda-functions: ['Jupyter Notebook']
Languages for fork TMeesters/lab-list-comprehensions: ['Jupyter Notebook']
Languages for fork TMeesters/lab-matplotlib-seaborn: ['Jupyter Notebook']
Languages for fork TMeesters/lab

In [ ]:
# Challenge 2: Count Commits

In [7]:
import requests
from datetime import datetime, timedelta

def count_commits_in_last_weeks_for_user(username, weeks=3):
    events_url = f"https://api.github.com/users/{username}/events"
    
    try:
        # Make the API request
        response = requests.get(events_url)
        response.raise_for_status()
        events = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return

    three_weeks_ago = datetime.now() - timedelta(weeks=weeks)

    commit_count = 0

    for event in events:
        if event.get('type') == 'PushEvent': 
            event_date = datetime.strptime(event['created_at'], "%Y-%m-%dT%H:%M:%SZ")
            if event_date >= three_weeks_ago:
                commit_count += len(event['payload']['commits'])

    print(f"Number of commits by {username} in the last {weeks} weeks: {commit_count}")

github_username = "TMeesters"

count_commits_in_last_weeks_for_user(github_username, weeks=3)


Number of commits by TMeesters in the last 3 weeks: 15
